# Webpages

Webpages are built mainly with 3 tools:

    HTML: content (structure, headings, paragraphs, tables...)
    CSS: style (color, shape, size...)
    JavaScript: logic (clicks, popups, dynamic banners...)

## HTML
Basics

HTML code consists of `<tagged>` content.

HTML has a hierarchichal structure: parent tags, children tags, sibling tags:
```
<html>
    <head>
        <title>Page Title</title>
    </head>
    <body>
        <h1>My First Heading</h1>
        <p>My first paragraph.</p>
    </body>
</html>
```


### Tags

Tags may be classified in different groups, depending on the type of content they are expected to posess
```
heading: <h1>, <h2>, <h3>, <hgroup>...
phrasing: <b>, <img>, <sub>...
embedded: <audio>, <img>, <video>, <href>...
tabulated: <table>, <tr>, <tbody>...
sections: <header>, <section>, <article>, <div>...
metadata: <meta>, <title>, <script>...
```


### Attributes

This tag has no attributes `<div> Zapas Marca Joma X54 </div>`

Tags may have attributes. Here, the div tag has:

    a class attribute with value price-item
    an id attribute with value offer
```
<div class="price-item" id="offer"> Zapas Marca Joma X54 </div>
```
id attribute should be ***unique*** for a tag (no two tags should have same id)

class is not intended to be unique, it usually groups tags with similar behavior

Other frequently used tags are:
```
    dir
    lang
    style (not to be confused with <style> tag)
    title (not to be confused with <title> tag)
``` 


### Web scraping
Intro

When scraping, we want to filter tags by:
```
    tag name
    class
    id
    other attribute
```

Our browser Console is very useful for this: we can Inspect content in the web and find the corresponding piece of HTML code

We use requests library to bring the HTML content to our Python script

We use `Beautiful Soup` library to easily navigate through the HTML in Python


In [53]:
# Descargamos la librería bs4
!pip install beautifulsoup4

In [9]:
# Otras librerías para poder interactuar con una web
import requests
from bs4 import BeautifulSoup

In [55]:
# Probamos a navegar en una web
url = "https://www.elpais.com"

# Lanzamos la request
response = requests.get(url)
response.status_code

200

In [56]:
# Comprobamos el contenido
response.content

b'<!DOCTYPE html><html lang="es-ES"><head><link rel="preconnect" href="//static.elpais.com"/><link rel="preconnect" href="//ep00.epimg.net"/><link rel="preconnect" href="//imagenes.elpais.com"/><link rel="preload" as="script" href="//ep00.epimg.net/js/prisa/user.min.js?i=1"/><link rel="preconnect" href="//www.googletagservices.com"/><link rel="preconnect" href="//ping.chartbeat.net"/><link rel="preconnect" href="//cdn.krxd.net"/><link rel="preconnect" href="//assets.adobedtm.com"/><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Bold.woff2" as="font" type="font/woff2" crossorigin=""/><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/majrit/majrit-text/Majrit-Text-Roman.woff2" as="font" type="font/woff2" crossorigin=""/><link rel="preload" href="https://static.elpais.com/dist/resources/fonts/marcin-ant-b/marcinantb-regular-webfont.woff2" as="font" type="font/woff2" crossorigin=""/><link rel="preload" href="htt

In [57]:
# Check del type
type(response.content)

bytes

> Tal como ocurre para el fichero `json` es un formato serializado, necesitaremos deserializarlo.
>

In [58]:
# Utilizamos soup para este labor de deserialización
soup = BeautifulSoup(response.content)
type(soup)

bs4.BeautifulSoup

> si lanzamos esta consulta dentro del inspector del navegador obtenemos los resultados linkables en color rojo

```
document.querySelectorAll('a').forEach(elm => elm.style.background = 'red')
```

In [59]:
# Enviamos una petición similar con bs4 y recogemos los anchor text ('a')
posibles_elementos_a = soup.select("a")
len(posibles_elementos_a)

583

In [60]:
# Enviamos una petición similar con bs4 y recogemos los h2 de estos anchor text
posibles_elementos_h2 = soup.select("h2 a")
len(posibles_elementos_h2)

143

In [61]:
# Enviamos una petición similar con bs4 y recogemos los h2
posibles_elementos_h1 = soup.select("h1")
len(posibles_elementos_h1)

1

In [62]:
type(posibles_elementos_h1) # el formato output es una list

bs4.element.ResultSet

In [63]:
# Para extraer el contenido textual recorremos elementos del listado y seleccionaremos text
posibles_elementos_h2[0].text

'El PP se conjura para enterrar la etapa de Casado en torno a Feijóo'

In [64]:
# Sacamos todos los textos del formato h2 y lo guardamos en una nueva lista
lista_h2 = [h2.text for h2 in posibles_elementos_h2]
len(lista_h2)

143

In [65]:
lista_h2[:5]

['El PP se conjura para enterrar la etapa de Casado en torno a Feijóo',
 'Gamarra: “Putin es un problema pero la causa de nuestra crisis es Sánchez”',
 'Aznar: “Pablo, donde quiera que esté, gracias”',
 'Ayuso: “Este congreso es la respuesta a una crisis que nunca debió existir”',
 'Videoanálisis | ¿Es el fin de la crisis del Partido Popular?']

### Realizamos una extracción de los resultados de búsqueda en Google

In [66]:
# El primer paso para realizar scraping es crear un agente del navegador
USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

> Queremos extraer los resultados de unos términos de búsqueda

In [67]:
# Creamos una lista de
kwterms = ['python', 'data science','data analytics', 'big data']  

In [68]:
# Creamos la funciones de recogida de los resultados
def fetch_results(search_term, num_results, lang_code, country_code, time_results):
    assert isinstance(search_term, str), 'Search term tiene que ser una string'
    assert isinstance(num_results, int), 'Número de resultado tiene que ser entero'
    escaped_search_term = search_term.replace(' ', '+')

    google_url = 'https://www.google.es/search?q={}&num={}&hl={}&cr=country{}&as_qdr={}'.format(escaped_search_term, 
                                                                                                 num_results,
                                                                                                 lang_code,
                                                                                                 country_code,
                                                                                                 time_results)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()

    return search_term, response.text

> Definimos una función para extraer de la página de resultados de Google los elementos solicitados

In [83]:
# Parse Results for Google Search
def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')

    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:

        link = result.find('a', href=True)
        title = result.find('h3')
        description = result.find('span', attrs={'class': 'MUxGbd wuQ4Ob WZ8Tjf'})
        if link and title:
            link = link['href']
            title = title.get_text()
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'rank': rank, 'title': title, 
                                      'description': description, 'link':link})
                rank += 1
    return found_results

In [70]:
# Guardamos la información
import pandas as pd

def store_data(keyword, rank, title, description, link):
    google_se = pd.DataFrame({
        "keyword":keyword,
        "rank":rank,
        "title":title,
        "description":description,
        "link":link   
    })
    return google_se



In [71]:
# Recoger la información y almacenar en una base de datos

def on_data(data):
    #This is the meat of the script...it connects to your mongoDB and stores the search results
    try:
        # Create dataJson
        for i in data:
                   
        #grab the keys and values
            keyword = i['keyword']
            rank = i['rank']
            title = i['title']
            description = i['description']
            link = i['link']
           
            #insert the data into the MySQL database
            store_data(keyword, rank, title, description, link)

    except Exception as e:
        print(e)



In [79]:
# Scraping Web function for Google SEARCH only 50 results, spanish, Spain, last 1 day
def scrape_web(search_term):
        keyword, html = fetch_results(search_term, 50, "es", "ES" ,"d") # last 1 day
        results = parse_results(html, keyword)
        return results



In [73]:
# # Scraping Web function for Google SEARCH only 1000 results, spanish, Spain, all results
# def scrape_web(search_term):
#         keyword, html = fetch_results(search_term, 1000, "es", "ES" ,"all") # all results
#         results = parse_results(html, keyword)
#         return results



In [84]:
# Testing googleScraping
data = []
results = scrape_web(kwterms[0])
#results = scrape_web('master data science')
for result in results:
    data.append(result)

In [85]:
len(results)

46

In [86]:
results

[{'keyword': 'python',
  'rank': 1,
  'title': 'Trabajo de Python en Barcelona - InfoJobs',
  'description': 'hace 10 horas — ',
  'link': 'https://www.infojobs.net/ofertas-trabajo/barcelona/python'},
 {'keyword': 'python',
  'rank': 2,
  'title': 'CT02 - PYTHON Developer (Inglés Alto) - LinkedIn',
  'description': 'hace 14 horas — ',
  'link': 'https://es.linkedin.com/jobs/view/ct02-python-developer-ingl%C3%A9s-alto-at-ingefor-2999666368'},
 {'keyword': 'python',
  'rank': 3,
  'title': '4.1. Acércate a Python | Cambiamos el mundo - Junta de ...',
  'description': None,
  'link': 'https://edea.juntadeandalucia.es/bancorecursos/file/653ee80d-2517-4e6b-938d-8fc23331967f/1/PRO_2BAC_REA_01_V01.zip/41_acrcate_a_python.html'},
 {'keyword': 'python',
  'rank': 4,
  'title': '4.1. Acércate a Python | Cambiamos el mundo - Junta de ...',
  'description': None,
  'link': 'https://edea.juntadeandalucia.es/bancorecursos/file/653ee80d-2517-4e6b-938d-8fc23331967f/1/PRO_2BAC_REA_01_V01.zip/41_acrcate

In [87]:
dataset = pd.DataFrame(results)
dataset

,keyword,rank,title,description,link
0,python,1,Trabajo de Python en Barcelona - InfoJobs,hace 10 horas —,https://www.infojobs.net/ofertas-trabajo/barce...
1,python,2,CT02 - PYTHON Developer (Inglés Alto) - LinkedIn,hace 14 horas —,https://es.linkedin.com/jobs/view/ct02-python-...
2,python,3,4.1. Acércate a Python | Cambiamos el mundo - ...,None,https://edea.juntadeandalucia.es/bancorecursos...
3,python,4,4.1. Acércate a Python | Cambiamos el mundo - ...,None,https://edea.juntadeandalucia.es/bancorecursos...
4,python,5,"4.2. ""Hola Python"" | Cambiamos el mundo - Junt...",None,https://edea.juntadeandalucia.es/bancorecursos...
5,python,6,Introducción a los dispositivos gemelos (Pytho...,hace 18 horas —,https://docs.microsoft.com/es-es/azure/iot-hub...
6,python,7,Building a Hash Table in Python and Thoughtful...,hace 20 horas —,https://www.ivoox.com/building-a-hash-table-in...
7,python,8,Clases de inteligencia artificial en python - ...,hace 15 horas —,https://es.wallapop.com/item/clases-de-intelig...
8,python,9,DATA Engineer / Data Governance - Tecnoempleo,hace 23 horas —,https://www.tecnoempleo.com/data-engineer-data...
9,python,10,Empezar con Anaconda Python - Byte Spider,hace 7 horas —,https://bytespider.eu/empezar-con-anaconda-pyt...


In [ ]:
import time

if __name__ == '__main__':
    
    data = []
    for keyword in kwterms:
        try:
            results = scrape_web(keyword)
            
            for result in results:
                data.append(result)
             
        except Exception as e:
            print(e)
        finally:
            time.sleep(60)
    #print(data)
    # Call function to store data
    on_data(data)



> Otro Ejemplo desde una página de wikipedia

In [1]:
# Ejemplo extraemos las informaciones de wikipedia
# Cargamos las librerías requests y pandas

import requests
import pandas as pd

In [2]:
url = "https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid"

# Lanzamos la requests
respuesta = requests.get(url)
respuesta.status_code

200

In [3]:
# Comprobamos el contenido
respuesta.content

b'<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Anexo:Municipios de la Comunidad de Madrid - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xc2\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"12ea14c4-fdd8-4221-b541-4c1f8cff9edf","wgCSPNonce":false,"wgCanonicalNamespace":"Anexo","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":104,"wgPageName":"Anexo:Municipios_de_la_Comunidad_de_Madrid","wgTitle":"Municipios de la Comunidad de Madrid","wgCurRevisionId":141671874,"wgRevisionId":141671874,"wgArticleId":22109,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Anexos:Municipios d

In [4]:
# Guardamos el contenido
codigoHTML = respuesta.text
codigoHTML

'<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Anexo:Municipios de la Comunidad de Madrid - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"12ea14c4-fdd8-4221-b541-4c1f8cff9edf","wgCSPNonce":false,"wgCanonicalNamespace":"Anexo","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":104,"wgPageName":"Anexo:Municipios_de_la_Comunidad_de_Madrid","wgTitle":"Municipios de la Comunidad de Madrid","wgCurRevisionId":141671874,"wgRevisionId":141671874,"wgArticleId":22109,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Anexos:Municipios de Esp

In [7]:
#!pip install lxml

In [5]:
# En pandas tenemos la función pd.read_html que nos permitirá leer este código
dataframe = pd.read_html(codigoHTML, header=0)
dataframe

[                    Nombre Población(2021)  Superficie (km²)[1]​  Mapa  \
 0               La Acebeda              55                  2206   NaN   
 1                  Ajalvir            4676                  1962   NaN   
 2        Alameda del Valle             246                  2501   NaN   
 3                 El Álamo            9908                  2225   NaN   
 4        Alcalá de Henares          195982                  8772   NaN   
 ..                     ...             ...                   ...   ...   
 174        Villar del Olmo            1997                  2762   NaN   
 175  Villarejo de Salvanés            7245                 11862   NaN   
 176   Villaviciosa de Odón          27 504                  6805   NaN   
 177  Villavieja del Lozoya             267                  2329   NaN   
 178              Zarzalejo            1596                  2063   NaN   
 
      Escudo  Altitud(msnm)[a]​[2]​  
 0       NaN                   1271  
 1       NaN          

In [6]:
# El response es una lista necesitamos extraer el primer elemento para mostrarlo como dataframe
dataframe[0]

,Nombre,Población(2021),Superficie (km²)[1]​,Mapa,Escudo,Altitud(msnm)[a]​[2]​
0,La Acebeda,55,2206,NaN,NaN,1271
1,Ajalvir,4676,1962,NaN,NaN,680
2,Alameda del Valle,246,2501,NaN,NaN,1104
3,El Álamo,9908,2225,NaN,NaN,608
4,Alcalá de Henares,195982,8772,NaN,NaN,587
...,...,...,...,...,...,...
174,Villar del Olmo,1997,2762,NaN,NaN,675
175,Villarejo de Salvanés,7245,11862,NaN,NaN,753
176,Villaviciosa de Odón,27 504,6805,NaN,NaN,661
177,Villavieja del Lozoya,267,2329,NaN,NaN,1060


> Creamos una lista con la TOP ten de los fugitivos más buscados por la FBI

In [21]:
import pandas as pd

In [27]:
# Url del sitio a buscar
url = 'https://www.fbi.gov/wanted/topten'

#your code 
fbi_html = requests.get(url).text
soup_fbi = BeautifulSoup(fbi_html, "html.parser")

buscados = [f.text.strip().replace('\n', '') for f in soup_fbi.find_all('h3', class_= 'title')]
buscados

[]

> Listar las imágenes de Walt Disney

In [32]:
# This is the url you will scrape in this exercise
url = 'https://en.wikipedia.org/wiki/Walt_Disney'

#your code
disney_html = requests.get(url).text
soup_disney = BeautifulSoup(disney_html, "html.parser")

enlaces = soup_disney.find_all('img')
images_src = [img['src'].replace('//', '') for img in soup_disney.findAll('img', class_='thumbimage')]
images_src


['upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Walt_Disney_envelope_ca._1921.jpg/220px-Walt_Disney_envelope_ca._1921.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Trolley_Troubles_poster.jpg/170px-Trolley_Troubles_poster.jpg',
 'upload.wikimedia.org/wikipedia/en/thumb/4/4e/Steamboat-willie.jpg/170px-Steamboat-willie.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/5/57/Walt_Disney_1935.jpg/170px-Walt_Disney_1935.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Walt_Disney_Snow_white_1937_trailer_screenshot_%2813%29.jpg/220px-Walt_Disney_Snow_white_1937_trailer_screenshot_%2813%29.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/1/15/Disney_drawing_goofy.jpg/170px-Disney_drawing_goofy.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/1/13/DisneySchiphol1951.jpg/220px-DisneySchiphol1951.jpg',
 'upload.wikimedia.org/wikipedia/commons/thumb/8/8c/WaltDisneyplansDisneylandDec1954.jpg/220px-WaltDisneyplansDisneylandDec1954.jpg',
 'upload.wikimedia.org/wiki

In [33]:
len(images_src)

13

> Buscamos los últimos 20 terremtos por (fecha, hora, lat, lon, region)

In [34]:
url = 'https://www.emsc-csem.org/Earthquake/'

ea_html = requests.get(url).text
soup_ea = BeautifulSoup(ea_html, "html.parser")

data = []
table = soup_ea.find_all('tr', class_= 'normal')




,date,time,latitude,longitude,region
0,2022-04-02,12:30:46.7,31.68 N,104.44 W,WESTERN TEXAS
1,2022-04-02,12:25:23.5,39.00 S,174.69 E,NORTH ISLAND OF NEW ZEALAND
2,2022-04-02,12:24:58.0,6.49 S,129.92 E,BANDA SEA
3,2022-04-02,12:22:29.0,7.47 S,106.88 E,"JAVA, INDONESIA"
4,2022-04-02,12:21:08.0,9.47 S,117.70 E,"SUMBAWA REGION, INDONESIA"
